In [1]:
import os
os.chdir('..')

from util import hand_to_str
from deck52 import random_deal
import binary
from nn.models import Models
from sample import Sample

from nn.bid_info import BidInfo
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


models = Models.from_conf(conf.load('./config/default_tf1x.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default_tf1x.conf'))  # Load sampling strategies

Loading config file d:\GitHub\ben\src\./config/default_tf1x.conf



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/contract/contract-193200
INFO:tensorflow:Restoring parameters from ..\models/GIB/binfo_V2-1420000
INFO:tensorflow:Restoring parameters from ..\models/lead/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\models/lead/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/single_dummy/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/pl

Loading config file d:\GitHub\ben\src\./config/default_tf1x.conf


In [2]:
auction = ['4C']
n_steps = binary.calculate_step_bidding_info(auction)
nesw_i = 1
vuln = [True, False]
hand = binary.parse_hand_f(32)("AJT85.AKT.K63.K8")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, models)


In [3]:
p_hcp, p_shp = models.binfo_model.pred_fun(A)
p_hcp = p_hcp.reshape((-1, n_steps, 3))[:, -1, :]
p_shp = p_shp.reshape((-1, n_steps, 12))[:, -1, :]

def f_trans_hcp(x): return 4 * x + 10
def f_trans_shp(x): return 1.75 * x + 3.25

p_hcp = f_trans_hcp(p_hcp)
p_shp = f_trans_shp(p_shp)

c_hcp = p_hcp.copy()
c_shp = p_shp.copy()

print(f"c_hcp:{c_hcp}")
print(f"c_shp:{c_shp}")


c_hcp:[[6.39 7.7  7.43]]
c_shp:[[3.38 3.95 3.65 2.02 3.44 4.03 3.74 1.78 1.47 1.99 1.92 7.74]]


In [4]:
auction = ['PAD_START', '2C','4N']
n_steps = binary.calculate_step_bidding_info(auction)
n_samples = 10
ns = -1
ew = -1
nesw_i = 3
vuln = [True, False]
hand = binary.parse_hand_f(32)("AJT8653.J.962.92")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, models)
p_hcp, p_shp = models.binfo_model.pred_fun(A)
c_hcp = (lambda x: 4 * x + 10)(p_hcp.copy())
print(c_hcp)


[[ 4.45 22.22  7.34]]


In [5]:
auction = ['PAD_START', '2C','4N','PASS']
n_steps = binary.calculate_step_bidding_info(auction)
n_samples = 10
nesw_i = 0
vuln = [True, False]
hand = binary.parse_hand_f(32)("K4.T9876532.54.6")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, models)
p_hcp, p_shp = models.binfo_model.pred_fun(A)
c_hcp = (lambda x: 4 * x + 10)(p_hcp.copy())
print(c_hcp)


[[20.93 11.66  2.67]]
